# SHAP Explain Function

## Version 1.0

In [ ]:
import numpy as np
import shap
from typing import Optional, Tuple

def shap_feature_importance(
    model,
    x: np.ndarray,
    background: np.ndarray,
    normalize: bool = False
) -> Tuple[np.ndarray, float]:
    # Ensure proper shapes
    x = np.asarray(x, dtype=float).reshape(1, -1)
    background = np.asarray(background, dtype=float)

    # Build generic SHAP explainer
    explainer = shap.Explainer(model, background)

    # Compute SHAP explanation
    explanation = explainer(x)

    # Extract SHAP values
    values = explanation.values
    base = float(explanation.base_values.mean())

    # Handle classification (multi-output)
    if values.ndim == 3:
        # Choose output with largest predicted score if available
        if hasattr(model, "predict_proba"):
            pred = model.predict_proba(x)[0]
            class_idx = int(np.argmax(pred))
        else:
            class_idx = 0
        values = values[0, class_idx, :]
    else:
        values = values[0, :]

    scores = values.astype(float)

    # Optional normalization
    if normalize and np.std(scores) > 0:
        scores = (scores - np.mean(scores)) / np.std(scores)

    return scores, base


## Version 2.0

In [ ]:
import numpy as np
import shap
from typing import Optional, Tuple, Callable


def _score_fn(model) -> Callable[[np.ndarray], np.ndarray]:
    # same idea as your helper
    if hasattr(model, "predict_proba"):
        return lambda X: np.asarray(model.predict_proba(X))[:, 1]
    if hasattr(model, "decision_function"):
        def f(X):
            z = np.asarray(model.decision_function(X)).reshape(-1)
            return 1.0 / (1.0 + np.exp(-z))
        return f
    return lambda X: np.asarray(model.predict(X)).reshape(-1).astype(float)


def shap_feature_importance(
    model,
    x: np.ndarray,
    background: np.ndarray,
    normalize: bool = False,
) -> Tuple[np.ndarray, float]:
    # Ensure shapes
    x = np.asarray(x, dtype=float).reshape(1, -1)
    background = np.asarray(background, dtype=float)

    # Build scoring function
    scorer = _score_fn(model)

    # Wrap scorer for SHAP
    def f_shap(X):
        return scorer(np.asarray(X, dtype=float))

    # Build explainer
    explainer = shap.Explainer(f_shap, background)

    # Explain this sample
    explanation = explainer(x)

    # Scores explained
    values = explanation.values[0, :].astype(float)
    scores = values

    # SHAP base value
    base_val = np.asarray(explanation.base_values).reshape(-1)[0]
    base = float(base_val)

    # Optional normalization
    if normalize and np.std(scores) > 0:
        scores = (scores - np.mean(scores)) / np.std(scores)

    return scores, base